In [ ]:
#賠償金未繳庫情形
import pandas as pd
pd.options.display.max_colwidth=0 #顯示全部字串
#付款憑單-集中支付檔-cetralized payment
#cp = pd.read_csv(r'D:\hlao\會計資訊檔案\ais\108_01_06\資料匯出-20190809143644-付款憑單明細檔.csv')
#r = pd.read_excel(r'D:\hlao\會計資訊檔案\ais\108_01_06\資料匯出-20190809143644-收入傳票明細檔.xlsx')
#ap=pd.read_csv(r'D:\hlao\會計資訊檔案\ais\108_01_06\資料匯出-20190809143644-請購單明細檔.csv')

p106 = pd.read_csv(r'D:\hlao\會計資訊檔案\ais\106\資料匯出-20190812095742-明細分類帳.csv')
p107 = pd.read_csv(r'D:\hlao\會計資訊檔案\ais\107\資料匯出-20190615134322-明細分類帳.csv')
p108 = pd.read_csv(r'D:\gdrive\會計資訊檔案\ais\108_01_07\資料匯出-20190827103901-明細分類帳.csv')
p = pd.concat([p106,p107,p108],axis=0)
p['傳票年份']=round(p['傳票日期']/10000,0).astype(int)
p.columns

In [156]:
# 106至108年捐款科目支用比率未達1成明細
po13y = pd.merge(po3y, p13y, how='left', left_on=['機關名稱','子目名稱'], right_on = ['機關名稱','子目名稱'])
po13y["('支出額', 106)"].fillna(0, inplace=True)
po13y["('支出額', 107)"].fillna(0, inplace=True)
po13y["('支出額', 108)"].fillna(0, inplace=True)
po13y["106支用比率"] = po13y["('支出額', 106)"]/po13y["('期初餘額', 106)"]
po13y["107支用比率"] = po13y["('支出額', 107)"]/po13y["('期初餘額', 107)"]
po13y["108支用比率"] = po13y["('支出額', 108)"]/po13y["('期初餘額', 108)"]
po13y["106支用比率"].fillna(0, inplace=True)
po13y["107支用比率"].fillna(0, inplace=True)
po13y["108支用比率"].fillna(0, inplace=True)
po13y
po13y[(po3y['子目名稱'].str.contains("非政府機關委託|非政府機關補助|助學金|發展教育基金|教育發展|獎學金|獎助金")
       |po3y['子目名稱'].str.contains("畢業|教育儲蓄專戶|富邦基金會學童認養金|慈善|贊助|民間團體委託|捐贈|捐資興學|捐助|同濟會|急難慰問金")
       |po3y['子目名稱'].str.contains("玉山文教基金會|仁愛|指定用途")      
       )
     & (po13y["('期初餘額', 108)"] > 0)
     & (po13y["108支用比率"] < 0.1)
     & (po13y["107支用比率"] < 0.1)
     & (po13y["106支用比率"] < 0.1)
     ].to_csv("low_used_donations.csv")


In [ ]:
# 前2年核有期初餘額惟未有支用數之代收代付或代辦、保管款科目明細
po13y = pd.merge(po3y, p13y, how='left', left_on=['機關名稱','子目名稱'], right_on = ['機關名稱','子目名稱'])
po13y[["('支出額', 106)","('支出額', 107)","('支出額', 108)"]].fillna(0, inplace=True)
po13y[  (po13y["('期初餘額', 108)"] > 0)
      & (po13y["('期初餘額', 107)"] > 0)
      & (po13y["('支出額', 108)"]  == 0)
      & (po13y["('支出額', 107)"]  == 0)
      & (~po3y['子目名稱'].str.contains("非政府機關委託|非政府機關補助|助學金|發展教育基金|教育發展|獎學金|獎助金")
        &~po3y['子目名稱'].str.contains("畢業|教育儲蓄專戶|富邦基金會學童認養金|慈善|贊助|民間團體委託|捐贈|捐資興學|捐助|同濟會|急難慰問金")
        &~po3y['子目名稱'].str.contains("玉山文教基金會|仁愛基金|指定用途")
        &~po3y['子目名稱'].str.contains("代扣|勞保|勞退|退休|健保|退撫|所得稅|公保|預付|代扣款|補充保費|值勤費")         
        )
     ]

In [ ]:
#106至108年期間應付代收子目代辦經費支出紀錄
p1 = p[p['子目名稱'].str.contains('應付代收',na=False)
      &p['借貸方註記']==1
      ][['機關名稱','總帳科目', '子目代碼', '子目名稱', '傳票年份', '摘要', '借貸方註記', '金額',
         '來源傳票號碼', '經費來源', '經費來源名稱'
        ]
       ].sort_values(['機關名稱'],ascending=[False])
p1 = p1.rename({'金額': '支出額'}, axis='columns')
p13y = pd.DataFrame(pd.pivot_table(p1
                                 ,index=['機關名稱','子目名稱']
                                 ,columns=['傳票年份'],values=['支出額'],fill_value=0
                                 ).astype('int').to_records()
                   )
p13y

In [ ]:
#106至108年期間應付代收子目代辦經費期初金額3年均大於1000元者、排除捐款、保管款、代收代付、代扣款、學生代收代辦費用、午餐、基金來源
#列出各年份應付代收子目期初金額
po = p[p['子目名稱'].str.contains('應付代收',na=False)
 &p['摘要'].str.contains('結轉|上期|結帳',na=False)
 ][['機關名稱','總帳科目', '子目代碼', '子目名稱', '傳票年份', '摘要', '借貸方註記', '金額',
   '來源傳票號碼', '經費來源', '經費來源名稱'
   ]
].sort_values(['機關名稱'],ascending=[False])
po = po.rename({'金額': '期初餘額'}, axis='columns')
po3y = pd.DataFrame(pd.pivot_table(po
                                 ,index=['機關名稱','子目名稱']
                                 ,columns=['傳票年份'],values=['期初餘額'],fill_value=0
                                 ).astype('int').to_records()
                  )
po3y[(~po3y['子目名稱'].str.contains("非政府機關委託|非政府機關補助|助學金|發展教育基金|教育發展|獎學金|獎助金")
    &~po3y['子目名稱'].str.contains("畢業|教育儲蓄專戶|富邦基金會學童認養金|慈善|贊助|民間團體委託|捐贈|捐資興學|捐助|同濟會|急難慰問金")
    &~po3y['子目名稱'].str.contains("玉山文教基金會")
    &~po3y['子目名稱'].str.contains("學生繳費|幼兒園|幼稚園|簿本費|教科書")     
    &~po3y['子目名稱'].str.contains("午餐") 
    &~po3y['子目名稱'].str.contains("合作社|資源回收|場地設施使用費|租金收入|利息收入|收回薪津") #基金來源
    &~po3y['子目名稱'].str.contains("押金|保固金|保證金")
    &~po3y['子目名稱'].str.contains("代扣|勞保|勞退|退休|健保|退撫|所得稅|公保|預付|代扣款|補充保費|值勤費")
    &~po3y['子目名稱'].str.contains("薪資暫存")
    )
   &(po3y["('期初餘額', 106)"]>1000)
   &(po3y["('期初餘額', 108)"]>1000)
   &(po3y["('期初餘額', 107)"]>1000)
   ].sort_values(['機關名稱', '子目名稱', "('期初餘額', 108)"], ascending=[False, False, False])

In [ ]:
p[p['子目名稱'].str.contains('應付代收款－代收代付－其他',na=False)
 &p['機關名稱'].str.contains('光復國民小學',na=False)
 ][['機關名稱','總帳科目', '子目代碼', '子目名稱', '傳票年份', '摘要', '借貸方註記', '金額',
    '來源傳票號碼', '經費來源', '經費來源名稱'
   ]]

In [ ]:
#應付代收款項下校務基金用途明細
# general donation
gd = p[((p['子目名稱'].str.contains('校務發展|校務基金|校務推動|校務推動費|校務經費',na=False))
       |(p['摘要'].str.contains('校務發展|校務基金|校務推動|校務推動費|校務經費',na=False)))
      &~(p['摘要'].str.contains('上期結轉',na=False))
      &(p['借貸方註記']==1)
      &(p['總帳科目'].str.contains('2123'))
      ]
gd[['機關名稱', '傳票日期', '傳票類別名稱', '傳票號碼', '總帳科目', '子目名稱', "摘要",'借貸方註記','金額']].to_csv("donation_for_general_usage.csv")

In [ ]:
#應付代收款項下校務基金來源明細
# general donation
gd = p[((p['子目名稱'].str.contains('校務發展|校務基金|校務推動|校務推動費|校務經費',na=False))
       |(p['摘要'].str.contains('校務發展|校務基金|校務推動|校務推動費|校務經費',na=False)))
      &~(p['摘要'].str.contains('上期結轉',na=False))
      &(p['借貸方註記']==2)
      &(p['總帳科目'].str.contains('2123'))
      ]
gd[['機關名稱', '傳票日期', '傳票類別名稱', '傳票號碼', '總帳科目', '子目名稱', "摘要",'借貸方註記','金額']].to_csv("general_donation_source.csv")

In [ ]:
#代收代付校務發展基金機關收支金額
pd.pivot_table(gd
              ,index=['機關名稱','子目名稱']
              ,columns=['借貸方註記']
              ,values=['金額']
              ,fill_value=0
              ).astype('int')

In [ ]:
#列出各年份應付代收子目期初金額
p[p['子目名稱'].str.contains('應付代收',na=False)
 &p['摘要'].str.contains('結轉|上期|結帳',na=False)
 ][['機關名稱','總帳科目', '子目代碼', '子目名稱', '傳票年份', '摘要', '借貸方註記', '金額',
   '來源傳票號碼', '經費來源', '經費來源名稱'
   ]
].sort_values(['機關名稱'],ascending=[False])
p3y = pd.DataFrame(pd.pivot_table(p
                                 ,index=['機關名稱','子目名稱']
                                 ,columns=['傳票年份'],values=['金額'],fill_value=0
                                 ).astype('int').to_records()
                  )

In [ ]:
#106至108年期間應付代收子目期初金額逐年遞增者、排除捐款、補助
p3y[(~p3y['子目名稱'].str.contains("非政府機關委託補助|足球隊補助費|獎助學金|捐款|教職員工午餐費|拾金不昧|法院執行|幼兒午餐|值日夜經費|教師事病假代課費|值勤薪資|幼兒園-課後留園|幼兒園學生平安保險費|幼兒園學生餐點費|短代外聘教師年終獎金|導師費|優惠存款|臨時人員薪資|勞工退休|法院強制命令|代收代付-所得|代課鐘點費|加班費|兒午餐費|幼兒園材料費|幼兒園-保險費|幼兒園平安保險費|幼兒園-材料費|幼兒園午餐費|幼兒園-點心午餐費|健保|勞保|公保|退輔基金|休假補助款|所得稅|退撫基金|補充保費|薪資所得|勞退|退休準備|活動費|保費|點心費"))
   &(p3y["('金額', 106)"]>0)
   &(p3y["('金額', 108)"]>p3y["('金額', 107)"])
   &(p3y["('金額', 107)"]>p3y["('金額', 106)"])
   ].sort_values(['子目名稱', '機關名稱', "('金額', 108)"], ascending=[False, False, False])

In [ ]:
r[r['摘要'].str.contains('賠償收入繳庫',na=False)
 ][['機關名稱','傳票號碼', '傳票明細序號', '傳票類別代碼', '傳票類別名稱', '子目代碼', '子目名稱'
  ,'分支計畫代碼', '分支計畫名稱', '用途別代碼', '預計轉入科目代碼', '預計轉入子目代碼', '預計轉入子目名稱', '委受名稱'
  ,'摘要', '貸方金額', '原始憑證種類', '原始憑證號數', '貸方科目代碼', '經費來源', '經費來源名稱']
  ]

In [ ]:
ap[(ap['請購金額']==85289)]

In [ ]:
ap[(ap['機關名稱'].str.contains(r"澔學",na=False))]

In [ ]:
#服裝費支付員工活動服裝
ap[(ap['用途代碼'].str.contains('325'))
  &(ap['摘要'].str.contains("服裝"))        
  ][['機關名稱','總帳科目代碼', '子目名稱', '分支計畫名稱','用途代碼', '摘要', '請購金額']]

In [ ]:
#以公款支付罰款
cp[cp['摘要'].str.contains('罰|滯|違|未足額進用')][['摘要']]

In [ ]:
#是否以非公關費支付餐敘、禮品
cp[(cp['摘要'].str.contains(r"招待|禮|餐"))
   &(~cp['摘要'].str.contains('學校午餐|貧困午餐|稽查|接力|賽|國手|白禮袋|廚工|代扣|設備|餐飲科|明禮|餐廳|慈輝|教室|誤餐|夜輔|廚房|典禮|學生|午餐秘書|姜禮誠|日誌'))
   &(~cp['用途別代嗎'].str.contains('291|27F'))
   ][['機關名稱','借方科目碼', '分支計畫名稱','用途別代嗎','摘要', '實付數', '預計轉入科目代碼']]

In [ ]:
ap.columns

In [ ]:
#是否以非公關費支付餐敘、禮品
ap1 = ap[(~ap['用途代碼'].str.contains('291|27F'))
        &(ap['摘要'].str.contains("招待|禮"))        
        ]
ap2[(~ap2['摘要'].str.contains('學校午餐|書法營|畢業|門鎖|試辦計畫|白禮袋|家長|委員|期末|貧困午餐|兒童節|幼兒園|園遊會|稽查|活動|木工班|明禮|禮班|禮堂|接力|賽|國手|廚工|代扣|設備|體育班|義工|志工|餐飲科|餐廳|慈輝|教室|誤餐|夜輔|廚房|典禮|學生|午餐秘書|姜禮誠|日誌'))
  ][['機關名稱','總帳科目代碼', '子目名稱', '分支計畫名稱','用途代碼', '摘要', '請購金額']]

In [ ]:
#工程管理費超支
p1 = p[p['摘要'].str.contains('花崗國中Q棟',na=False)]
p1[~p1['摘要'].str.contains('保證')
  ][['機關名稱', '年度'
    , '教育部機關代碼', '傳票類別名稱', '傳票號碼', '序號', '借貸方註記', '傳票日期', '金額'
    ,'摘要', '總帳科目', '用途別代碼', '分支計畫代碼', '分支計畫名稱', '子目代碼', '子目名稱'
    , '過帳日期', '經費來源', '經費來源名稱']]